In [1]:
%matplotlib inline
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
from typing import Optional, Tuple, Dict
import MDAnalysis as mda
from MDAnalysis.analysis.base import AnalysisFromFunction
import numpy as np
import pandas as pd
import os
from polyphys.manage.typer import ParserT
from polyphys.manage.parser import SumRuleCyl, TransFociCyl, TransFociCub
from polyphys.manage.organizer import invalid_keyword, sort_filenames
from polyphys.analyze import clusters
from polyphys.probe.prober import stamps_report
import warnings

from MDAnalysis.analysis import polymer
from MDAnalysis.analysis import distances
from polyphys.manage.parser import HnsCub

### Probe explorations

In [ ]:
# analyzing bug files.
group = 'nucleoid'
lineage = 'whole'
save_to = './'
#path = "/Users/amirhsi/OneDrive - University of Waterloo/PhD Research/Jupyter/N200epshm29nh0ac2nc0l25dt0.005ndump2000adump5000ens1.ring"
path = "/Users/amirhsi_mini/research_data/hns_cubic-trjs/N*/N*"
nuc_pairs = glob(path + '/N*' + group + '*')
nuc_pairs = sort_filenames(
    nuc_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)

In [ ]:
topology = nuc_pairs[0][0]
trajectory = nuc_pairs[0][1]
lineage = 'whole'
save_to = './',
continuous = False
if (lineage == 'segment') & (continuous is False):
    warnings.warn(
        "lineage is "
        f"'{lineage}' "
        "and 'continuous' is "
        f"'{continuous}. "
        "Please ensure the "
        f"'{trajectory}' is NOT part of a sequence of trajectories.",
        UserWarning
    )
print("Setting the name of analyze file...")
sim_info = HnsCub(
    trajectory,
    lineage,
    'cubic',
    'nucleoid'
)
sim_name = sim_info.lineage_name + "-" + sim_info.group
print("\n" + sim_name + " is analyzing...\n")
# LJ time difference between two consecutive frames:
time_unit = sim_info.dmon * np.sqrt(
    sim_info.mmon * sim_info.eps_others)  # LJ time unit
lj_nstep = sim_info.ndump  # Sampling steps via dump command in Lammps
lj_dt = sim_info.dt
sim_real_dt = lj_nstep * lj_dt * time_unit
cell = mda.Universe(
    topology, trajectory, topology_format='DATA',
    format='LAMMPSDUMP', lammps_coordinate_convention='unscaled',
    atom_style="id resid type x y z", dt=sim_real_dt
    )
# slicing trajectory based the continuous condition
if continuous:
    sliced_trj = cell.trajectory[0: -1]
    n_frames = cell.trajectory.n_frames - 1
else:
    sliced_trj = cell.trajectory
    n_frames = cell.trajectory.n_frames
# selecting atom groups
bug = cell.select_atoms('resid 1')  # the bug
hns_pole = cell.select_atoms('type 2')  # the hns holes
hns_core = cell.select_atoms('type 3')  # the hns cores

# bond info
n_bonds = len(bug.bonds.indices)
bond_lengths_acc = np.zeros(n_bonds, dtype=np.float64)
cosine_corrs_acc = np.zeros(n_bonds, dtype=np.float64)
ljcut_coeff = 1.122462
hns_core_mon_cutoff = 0.5*(sim_info.dmon*ljcut_coeff + 0.2 + sim_info.dhns)
hns_pole_mon_cutoff = 0.5*(sim_info.dmon*ljcut_coeff + 0.2)
mon_mon_cutoff = 0.5*(sim_info.dmon*ljcut_coeff)
dist_array_core_contact = np.zeros((sim_info.nmon, sim_info.nhns), dtype=np.int64)
dist_array_pole_contact = np.zeros((sim_info.nmon, 2*sim_info.nhns), dtype=np.int64)
dist_array_mon_contact = np.zeros((sim_info.nmon, sim_info.nmon), dtype=np.int64)

In [ ]:
type(bug.positions)

In [ ]:
for _ in sliced_trj:
    # bug:
    dummy = distances.distance_array(bug, hns_core, box=cell.dimensions)
    dummy = np.asarray(dummy <= hns_core_mon_cutoff, dtype=int)
    dist_array_core_contact += dummy
    dummy = distances.distance_array(bug, hns_pole, box=cell.dimensions)
    dummy = np.asarray(dummy <= hns_pole_mon_cutoff, dtype=int)
    dist_array_pole_contact += dummy
    dummy = distances.distance_array(bug, bug, box=cell.dimensions)
    dummy = np.asarray(dummy <= mon_mon_cutoff, dtype=int)
    dist_array_mon_contact += dummy

dist_array_core_contact_mean = dist_array_core_contact / n_frames
dist_array_pole_contact_mean = dist_array_pole_contact / n_frames
dist_array_mon_contact_mean = dist_array_mon_contact / n_frames
np.save(
    save_to + sim_name + "distCoreMonMean.npy",
    dist_array_core_contact_mean
)
np.save(
    save_to + sim_name + "-distPoleMonMean.npy",
    dist_array_pole_contact_mean
)
np.save(
    save_to + sim_name + "distMonMonMean.npy",
    dist_array_pole_contact_mean
)

In [ ]:
%%time
# analyzing bug files.
group = 'nucleoid'
lineage = 'whole'
save_to = './'
probe_path = "/Users/amirhsi_mini/research_data/hns_cubic-trjs/N*/N*"
#probe_path = "/Users/amirhsi_mini/research_data/ns400nl5al5ml125ac1nc0l62dt0.005bdump2000adump5000ens3.ring"
nuc_pairs = glob(probe_path + '/N*' + group + '*')
nuc_pairs = organizer.sort_filenames(
    nuc_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (nuc_topo, nuc_trj) in nuc_pairs:
    prober.hns_nucleoid_cub_dist(
        nuc_topo,
        nuc_trj,
        lineage,
        save_to = save_to
    )

## Visualizing contacts:

In [ ]:
phys_attrs = {
    "Core": "distCoreMonMean",
    "Pole": "distPoleMonMean"
    
}
attr = "Pole"
dists = glob("./N200*"+phys_attrs[attr]+".npy")
dists = sorted(dists)
dists = dists[1:]
dist_lists = []
for dist in dists:
    dist_info = HnsCub(
        dist,
        'whole',
        'cubic',
        'nucleoid'
    )
    
    dist_lists.append([dist_info.nhns, dist_info.dcrowd, round(dist_info.phi_c_bulk,2), dist_info.nmon, dist_info.filepath])
dist_lists.sort(key=lambda dist: (dist[0], dist[2]))

In [ ]:
rc_params = {
    'mathtext.default': 'regular',
    #'text.usetex': True
}
sns.set_theme(
 #   context='paper',
    font='Times New Roman',
    font_scale=1.5,
    rc=rc_params
)
fig, axes = plt.subplots(nrows=5, ncols=2, sharey=True, figsize=(15,20))
#cbar_ax = fig.add_axes([.91, .2, .03, .6])
nmon = 200
for idx, (dist, ax) in enumerate(zip(dist_lists, axes.flatten(order='F'))):
    pole_data = np.load(dist[-1])
    ax_hmap = sns.heatmap(
        pole_data, ax=ax, cmap='rocket_r', vmin=0, vmax=0.03, # Pole
        #pole_data, ax=ax, cmap='rocket_r', vmin=0, vmax=0.045,  # Core
        #square=True
        #cbar_kws={'label': 'Normalized contacts'},
        #cbar= idx > 4,
        #cbar_ax= None if idx==0 else cbar_ax,
    )
    
    ax.set_xticks(np.arange(0.5,2*dist[0]+0.5,5))
    ax.set_xticklabels(np.arange(1,2*dist[0],5))
    #ax.set_xticks(np.arange(0.5,dist[0]+0.5,5))
    #ax.set_xticklabels(np.arange(1,dist[0],5))
    #ax.set_xticks(np.arange(0.5,dist[3]+0.5,30))
    #ax.set_xticklabels(np.arange(1,dist[3],30))
    ax.set_yticks(np.arange(0.5,dist[3],30))
    ax.set_yticklabels(np.arange(1,dist[3],30))
    if idx < 5:
        ax.set_ylabel('Monomer (index)')
    if idx % 5 == 4:
        ax.set_xlabel('Hns pole (index)')
    ax.set_title(fr'$n_{{hns}}={dist[0]}, \phi_c={dist[2]}, a_c={dist[1]}$')
fig.suptitle(
    r"Normalized number of contacts between monomers and hns " \
        + attr.lower() +  "s in $10^5$ counts"
)
fig.tight_layout(h_pad=1, w_pad=0)#rect=[0, 0, .9, 1])
fig.savefig("ContantMap-"+attr+"Mon.pdf")

# Visualizing persistance length

In [3]:
import scipy
def fit_exponential_decay(x, y):
    r"""Fit a function to an exponential decay

    .. math::  y = \exp\left(- \frac{x}{a}\right)

    Parameters
    ----------
    x, y : array_like
      The two arrays of data

    Returns
    -------
    a : float
      The coefficient *a* for this decay

    Notes
    -----
    This function assumes that data starts at 1.0 and decays to 0.0

    """
    def expfunc(x, a, b, c):
        return np.exp(-x**b/a) 

    a, b, c = scipy.optimize.curve_fit(expfunc, x, y)[0]

    return a, b, c

In [8]:
corrs = glob("./N200*-bondCosineCorrsVec.npy")
corrs = sorted(corrs)
corr_lists = []
for corr in corrs:
    corr_info = HnsCub(
        corr,
        'whole',
        'cubic',
        'nucleoid',
        'ring'
    )
    corr_lists.append([corr_info.nhns, corr_info.dcrowd, round(corr_info.phi_c_bulk,2), corr_info.nmon, corr_info.filepath])
corr_lists.sort(key=lambda dist: (dist[0], dist[2]))

/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Jupyter/PolyPhys/polyphys/manage/parser.py:2000: UserWarning: It is assumed that 'nc' is the last attribute shortkey in a lineage_name of types: 'ensemble', 'ensemble_long', 'whole', 'segment'.
  warnings.warn(convention_warning, UserWarning)


In [9]:
corr_lists

[[0,
  2.0,
  0.0,
  200,
  './N200epshm29nh0ac2nc0l25dt0.005ndump2000adump5000ens1.ring-nucleoid-bondCosineCorrsVec.npy'],
 [37,
  1.0,
  0.2,
  200,
  './N200epshm29nh37ac1nc47747l25dt0.005ndump2000adump5000ens1.ring-nucleoid-bondCosineCorrsVec.npy'],
 [37,
  1.0,
  0.3,
  200,
  './N200epshm29nh37ac1nc71620l25dt0.005ndump2000adump5000ens1.ring-nucleoid-bondCosineCorrsVec.npy']]

In [ ]:
fit_exponential_decay(bond_idx, cosine_corrs_acc/n_bonds/n_frames)

In [ ]:
corr_data = np.load(corr[-1])
bonds_per_lag = np.arange(n_bonds,0, -1)
corr_data = corr_data/bonds_per_lag
bond_idx = np.arange(corr[3])
per_length, b, c = fit_exponential_decay(bond_idx[:n_bonds//2], corr_data[:n_bonds//2])
bond_corr = np.exp(-c*bond_idx**b/per_length)
print(per_length, b, c)

In [ ]:
corr_data

In [ ]:
plt.plot(bond_idx, corr_data)
plt.plot(bond_idx, np.exp(-c*bond_idx**b/per_length))

In [15]:
np.arange(1, 201)

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [ ]:
rc_params = {
    'mathtext.default': 'regular',
    #'text.usetex': True
}
sns.set_theme(
 #   context='paper',
    font='Times New Roman',
    font_scale=1.5,
    rc=rc_params
)
fig, axes = plt.subplots(nrows=5, ncols=2, sharey=True, figsize=(15,20))
#cbar_ax = fig.add_axes([.91, .2, .03, .6])
nmon = 200
for idx, (corr, ax) in enumerate(zip(corr_lists, axes.flatten(order='F'))):
    corr_data = np.load(corr[-1])
    bond_idx = np.arange(1,corr[3]+1,1)
    ax_hmap = sns.lineplot(
        bond_idx, corr_data,
        #square=True
        #cbar_kws={'label': 'Normalized contacts'},
        #cbar= idx > 4,
        #cbar_ax= None if idx==0 else cbar_ax,
    )
    
    ax.set_xticks(np.arange(0.5,2*dist[0]+0.5,5))
    ax.set_xticklabels(np.arange(1,2*dist[0],5))
    #ax.set_xticks(np.arange(0.5,dist[0]+0.5,5))
    #ax.set_xticklabels(np.arange(1,dist[0],5))
    #ax.set_xticks(np.arange(0.5,dist[3]+0.5,30))
    #ax.set_xticklabels(np.arange(1,dist[3],30))
    #ax.set_yticks(np.arange(0.5,dist[3],30))
    #ax.set_yticklabels(np.arange(1,dist[3],30))
    #if idx < 5:
    #    ax.set_ylabel('Monomer (index)')
    #if idx % 5 == 4:
    #    ax.set_xlabel('Hns pole (index)')
    #ax.set_title(fr'$n_{{hns}}={dist[0]}, \phi_c={dist[2]}, a_c={dist[1]}$')
#fig.suptitle(
#    r"Normalized number of contacts between monomers and hns cores in $10^5$ #counts"
#)
#fig.tight_layout(h_pad=1, w_pad=0)#rect=[0, 0, .9, 1])
fig.savefig("PersistanceLength.pdf")

### slicing over different

In [ ]:

t_choice = [  # upper 1.03*10**8
    [3*10**6, 2.53*10**8, r'$\Delta t/\tau=25\times 10^7$'],
    [0.03*10**8, 1.03*10**8, r'$\Delta t/\tau=10\times 10^7$'],
    [0.53*10**8, 1.03*10**8, r'$\Delta t/\tau=5\times 10^7$'],
    [0.83*10**8, 1.03*10**8, r'$\Delta t/\tau=2\times 10^7$'],
    [0.93*10**8, 1.03*10**8, r'$\Delta t/\tau=1\times 10^7$'],
    [0.98*10**8, 1.03*10**8, r'$\Delta t/\tau=0.5\times 10^7$'],
    [1.02*10**8, 1.03*10**8, r'$\Delta t/\tau=0.1\times 10^7$']
]
t_choice = [  # upper 2.03*10**8
    [3*10**6, 2.53*10**8, r'$\Delta t/\tau=25\times 10^7$'],
    [1.03*10**8, 2.03*10**8, r'$\Delta t/\tau=10\times 10^7$'],
    [1.53*10**8, 2.03*10**8, r'$\Delta t/\tau=5\times 10^7$'],
    [1.83*10**8, 2.03*10**8, r'$\Delta t/\tau=2\times 10^7$'],
    [1.93*10**8, 2.03*10**8, r'$\Delta t/\tau=1\times 10^7$'],
    [1.98*10**8, 2.03*10**8, r'$\Delta t/\tau=0.5\times 10^7$'],
    [2.02*10**8, 2.03*10**8, r'$\Delta t/\tau=0.1\times 10^7$']
]
t_choice = [  # upper 1.53*10**8
    [3*10**6, 2.53*10**8, r'$\Delta t/\tau=25\times 10^7$'],
    [0.53*10**8, 1.53*10**8, r'$\Delta t/\tau=10\times 10^7$'],
    [1.03*10**8, 1.53*10**8, r'$\Delta t/\tau=5\times 10^7$'],
    [1.33*10**8, 1.53*10**8, r'$\Delta t/\tau=2\times 10^7$'],
    [1.43*10**8, 1.53*10**8, r'$\Delta t/\tau=1\times 10^7$'],
    [1.48*10**8, 1.53*10**8, r'$\Delta t/\tau=0.5\times 10^7$'],
    [1.52*10**8, 1.53*10**8, r'$\Delta t/\tau=0.1\times 10^7$']
]

In [ ]:
t_choice = [  # upper 1.53*10**8
    [0, 10*10**5, r'$\Delta t/\tau=25\times 10^7$'],
    [5*10**5, 10*10**5, r'$\Delta t/\tau=10\times 10^7$'],
    [1*10**5, 5*10**5, r'$\Delta t/\tau=5\times 10^7$'],
    [5*10**4, 10*10**4, r'$\Delta t/\tau=2\times 10^7$'],
    [2.5*10**4, 5*10**4, r'$\Delta t/\tau=1\times 10^7$'],
    [1.25*10**4, 2.5*10**4, r'$\Delta t/\tau=0.5\times 10^7$'],
    [0, 10**5, r'$\Delta t/\tau=0.1\times 10^7$']
]

In [ ]:
fig, axes = plt.subplots(7,1, figsize=(16,36))
fig.suptitle(r"$t_{final}/\tau=15.3\times 10^7$",y=0.98)
color_palette = 'flare'
#color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both',
    'xtick.top': True,
    'xtick.direction': 'in',
    'ytick.right': True,
    'ytick.direction': 'in',
}
font_family = 'sans-serif'
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
for ax, (t_i, t_f, dt) in zip(axes,t_choice):
    gyr_sliced = gyr.loc[(gyr['t']>=t_i)&(gyr['t']<=t_f),:]
    gyr_mean = np.round(gyr_sliced['gyr'].mean(),2)
    ax = sns.lineplot(
        data=gyr_sliced,
        x='t',
        y='gyr',
        ci=None,
        palette=color_palette,
        ax=ax,
        label=dt,
    )
    ax.axhline(
        gyr_mean, color='royalblue',
        label=rf'$\langle \sqrt{{R^2_g}}\rangle={gyr_mean}$', 
        lw=2, ls='-'
    )
    ax.legend(facecolor='white', loc='upper left', ncol=2)
    ax.set_ylabel(r'$\sqrt{R^2_g}/\sigma$')
    ax.set_xlabel(r'$t/\tau$')
fig.tight_layout(w_pad=1, h_pad=0)
plt.show()
fig.savefig("gyrTDna-slices.pdf")

### Semi-log to show reaching equilibrium

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16,18))
color_palette = 'flare'
save_to = './'
ext = 'pdf'
plot_context = 'talk'
font_scale = 2
height = 4
aspect = 2*1.618
ylabel_pad = 50
rc_params= {
    'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    'axes.grid': True,
    'axes.grid.axis': 'both',
    'axes.grid.which': 'both',
    'xtick.top': True,
    'xtick.direction': 'in',
    'ytick.right': True,
    'ytick.direction': 'in',
}
font_family = 'sans-serif'
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
axes[0] = sns.lineplot(
    data=gyr,
    x='t',
    y='gyr',
    ci=None,
    palette=color_palette,
    ax=axes[0]
)
fig.suptitle(r"$\Delta t_{sim}/\tau=28.7\times 10^7$",y=0.92)
axes[0].set_xlabel(r'$t/\tau$')
axes[0].axhline(gyr_mean, color='royalblue', label=r'$\langle \sqrt{R^2_g}\rangle$', 
                lw=2, ls='-')
axes[0].legend(facecolor='white', loc='upper right')
axes[0].set_ylabel(r'$\sqrt{R^2_g}/\sigma$')
axes[1].semilogx(
    gyr['t'],
    gyr['gyr']
)
axes[1].set_xlabel(r'$\log(t/\tau)$')
axes[1].axhline(gyr_mean, color='royalblue', label=r'$\langle \sqrt{R^2_g}\rangle$', 
                lw=2, ls='-')
axes[1].legend(facecolor='white', loc='upper right')
axes[1].set_ylabel(r'$\sqrt{R^2_g}/\sigma$')
plt.show()
fig.savefig("gyrTDna-semiLogX.pdf")